In [1]:
import equinox as eqx
import gym
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import optax
import sys
import timeit
import wandb

from argparse import Namespace
from functools import partial
from jax import grad, jit, vmap
from typing import Sequence, Tuple, Dict

from jax_learning.agents.rl_agents import RLAgent
from jax_learning.buffers.ram_buffers import NextStateNumPyBuffer
from jax_learning.buffers.utils import batch_flatten, to_jnp
from jax_learning.constants import DISCRETE, CONTINUOUS
from jax_learning.learners.reinforce import REINFORCE
from jax_learning.rl_utils import interact, evaluate

from jax_learning.models.policies import MLPGaussianPolicy

In [2]:
wandb.init(project="test_jax_rl", group="mountaincarcontinuous-reinforce_test")
wandb.define_metric("episodic_return", summary="max")

/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: chan. Use `wandb login --relogin` to force relogin
/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


In [3]:
cfg_dict = {
    # Environment setup
    "env": "MountainCarContinuous-v0",
    "seed": 0,
    "render": False,
    "clip_action": True,
    "max_action": 1.,
    "min_action": -1.,
    
    # Experiment progress
    "load_step": 0,
    "log_interval": 5000,
    
    # Learning hyperparameters
    "gamma": 0.99,
    "max_timesteps": 1000000,
    "update_frequency": 1000,
    "lr": 3e-4,
    "max_grad_norm": 10.,
    
    # Normalization
    "normalize_obs": True,
    "normalize_value": True,
    
    # Model architecture
    "hidden_dim": 128,
    "num_hidden": 2,
    
    # Evaluation
    "eval_cfg": {
        "max_episodes": 100,
        "seed": 1,
        "render": True,
        "clip_action": True,
        "max_action": 1.,
        "min_action": -1.,
    }
}
cfg = Namespace(**cfg_dict)
eval_cfg = Namespace(**cfg.eval_cfg)
wandb.config = cfg_dict

In [4]:
np.random.seed(cfg.seed)

In [5]:
env = gym.make(cfg.env)

/Users/chanb/work/personal_research/jax_learning/.venv/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [6]:
cfg.obs_dim = env.observation_space.shape
cfg.act_dim = env.action_space.shape
cfg.action_space = CONTINUOUS

In [7]:
cfg.h_state_dim = (1,)
cfg.rew_dim = (1,)

In [8]:
cfg

Namespace(env='MountainCarContinuous-v0', seed=0, render=False, clip_action=True, max_action=1.0, min_action=-1.0, load_step=0, log_interval=5000, gamma=0.99, max_timesteps=1000000, update_frequency=1000, lr=0.0003, max_grad_norm=10.0, normalize_obs=True, normalize_value=True, hidden_dim=128, num_hidden=2, eval_cfg={'max_episodes': 100, 'seed': 1, 'render': True, 'clip_action': True, 'max_action': 1.0, 'min_action': -1.0}, obs_dim=(2,), act_dim=(1,), action_space='continuous', h_state_dim=(1,), rew_dim=(1,))

In [9]:
cfg.buffer_rng = np.random.RandomState(cfg.seed)
cfg.env_rng = np.random.RandomState(cfg.seed)
cfg.agent_key, cfg.model_key = jrandom.split(jrandom.PRNGKey(cfg.seed), num=2)
eval_cfg.env_rng = np.random.RandomState(eval_cfg.seed)

In [10]:
POLICY = "policy"

buffer = NextStateNumPyBuffer(
    buffer_size=cfg.update_frequency,
    obs_dim=cfg.obs_dim,
    h_state_dim=cfg.h_state_dim,
    act_dim=(1,) if cfg.action_space == DISCRETE else cfg.act_dim,
    rew_dim=cfg.rew_dim,
    rng=cfg.buffer_rng,
)

model = {
    POLICY: MLPGaussianPolicy(obs_dim=cfg.obs_dim,
                              act_dim=cfg.act_dim,
                              hidden_dim=cfg.hidden_dim,
                              num_hidden=cfg.num_hidden,
                              key=cfg.model_key,
                              eps=1e-7)
}

opt_transforms = [
    optax.scale_by_rms(),
    optax.scale(-cfg.lr)
]
if cfg.max_grad_norm:
    opt_transforms.insert(0, optax.clip_by_global_norm(cfg.max_grad_norm))
opt = {
    POLICY: optax.chain(*opt_transforms)
}


learner = REINFORCE(model=model,
                    opt=opt,
                    buffer=buffer,
                    cfg=cfg)

agent = RLAgent(model=model,
                model_key=POLICY,
                buffer=buffer,
                learner=learner,
                key=cfg.agent_key)

/Users/chanb/work/personal_research/jax_learning/jax_learning/common.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.mean = np.zeros(shape, dtype=np.float)


In [11]:
%wandb

In [12]:
interact(env, agent, cfg)

In [13]:
evaluate(env, agent, eval_cfg)